In [ ]:
### This part of software should read the pre-processed files with valid data 
### and perform the spectral analisys.

### Steps
## 1. demultiplex data (avoid if not multiplexed)
## 2. deinterleaved channels
## 3. preform the FFT (should be performed in the GPU)
## 4. Calculate Signal Power

# %%writefile process.py

import os
import glob
import datetime
import mmap
import sys
import gc
import bitarray

def readmyheader(filename):
    import struct
    from collections import namedtuple   
    
    with open(filename, 'rb') as f:
        bytes=f.read(30)
        Header = namedtuple('Header', 'codename timetag_sec timetag_samp delay deltaID frequency quantization muxflag')
        header = Header._make(struct.unpack_from('>4sIIIIIIH',bytes))
        header = header.__dict__
    return header
    
def readdata(filename):
    
    stream = bitarray.bitarray()
    with open(filename, 'rb') as fh:
        fh.read(30) ### Avoid header
        stream.fromfile(fh)
    
    b=bitarray.bitarray(stream.length()/2)
    c=bitarray.bitarray(stream.length()/2)
    b.setall(False)
    c.setall(False)
    
    ch0 = bitarray.bitarray(stream.length())
    ch1 = bitarray.bitarray(stream.length())
    ch2 = bitarray.bitarray(stream.length())
    ch3 = bitarray.bitarray(stream.length())
    ch0.setall(False)
    ch1.setall(False)
    ch2.setall(False)
    ch3.setall(False)

    b, c = stream[::2], stream[1::2]
    ch0, ch2 = b[::2], b[1::2]
    ch1, ch3 = c[::2], c[1::2]
    
    print stream[0:128]
    print ch0[0:32]
    print ch1[0:32]
    print ch2[0:32]
    print ch3[0:32]

    print 'ch0 len',ch0.length()
    with open(filename+'_ch0.bin', 'ab') as f:
        ch0.tofile(f)
    print 'ch1 len',ch1.length()
    with open(filename+'_ch1.bin', 'ab') as f:
        ch1.tofile(f)
    print 'ch2 len',ch2.length()
    with open(filename+'_ch2.bin', 'ab') as f:
        ch2.tofile(f)
    print 'ch3 len',ch3.length()
    with open(filename+'_ch3.bin', 'ab') as f:
        ch3.tofile(f)
    
### E11=0,E21=1,E31=2,E12=3,E22=4,E32=5        
ll = ['/home/taller-dis1/Angel/ESA/testing/Archivos/OFF/test/MG11_NET4_2016_088_DD_E1_195710_cutfinal_OFF2.binfinal.bin',
      '/home/taller-dis1/Angel/ESA/testing/Archivos/OFF/test/MG12_NET4_2016_088_DD_E1_195714_cutfinal_OFF2.binfinal.bin',
      '/home/taller-dis1/Angel/ESA/testing/Archivos/OFF/test/MG13_NET4_2016_088_DD_E1_195718_cutfinal_OFF2.binfinal.bin',
      '/home/taller-dis1/Angel/ESA/testing/Archivos/OFF/test/MG11_NET4_2016_088_DD_E2_195710_cutfinal_OFF2.binfinal.bin',
      '/home/taller-dis1/Angel/ESA/testing/Archivos/OFF/test/MG12_NET4_2016_088_DD_E2_195714_cutfinal_OFF2.binfinal.bin',
      '/home/taller-dis1/Angel/ESA/testing/Archivos/OFF/test/MG13_NET4_2016_088_DD_E2_195718_cutfinal_OFF2.binfinal.bin']


for i in ll:
    filename=i
    header=readmyheader(filename)
    print header
    readdata(filename)
    

OrderedDict([('codename', 'AACM'), ('timetag_sec', 72539), ('timetag_samp', 17484464), ('delay', 4534), ('deltaID', 68), ('frequency', 176), ('quantization', 16), ('muxflag', 1)])
bitarray('10101010101110100100010000101110101010010000000011001101111000000000000000001011010100011100111001110000001010010110000001000000')
bitarray('11110001110011100001001100010000')
bitarray('00001101000011100000101110001010')
bitarray('11110011100000100001000110101000')
bitarray('00100000010001000001110010010000')
ch0 len 178181824
ch1 len 178181824
ch2 len 178181824
ch3 len 178181824
OrderedDict([('codename', 'AACM'), ('timetag_sec', 72539), ('timetag_samp', 17492224), ('delay', 4534), ('deltaID', 24), ('frequency', 176), ('quantization', 16), ('muxflag', 1)])
bitarray('00100010000001100111011001010110011110101110101100000110001000001001100110011001000001101000000010100010101100011111000101010000')
bitarray('00000000011100001111001010101000')
bitarray('00011111101001000000010000001010')
bitarray('110111

In [27]:
%%writefile spectrum.py

import numpy as np
import pyqtgraph as pg
# import pyaudio
from PyQt4 import QtCore, QtGui

import sys

import pycuda.autoinit
import pycuda.gpuarray as gpuarray
import numpy as np

import skcuda.fft as cu_fft

FS = 99183 #Hz

CHUNKSZ = 1024 #samples


class recorder():
    def __init__(self, signal):
        self.signal = signal
#         self.p = pyaudio.PyAudio()
#         self.stream = self.p.open(format=pyaudio.paInt16,
#                             channels=1,
#                             rate=FS,
#                             input=True,
#                             frames_per_buffer=CHUNKSZ)

    def read(self):
#         data = self.stream.read(CHUNKSZ)        
#         y = np.fromstring(data, 'int16')
        ix = np.arange(CHUNKSZ)
        pure = np.sin(20000*np.pi*ix/float(CHUNKSZ/2))*20000
        noise = np.random.normal(0, 1, CHUNKSZ)
        y = pure + noise
        self.signal.emit(y)

#     def close(self):
#         self.stream.stop_stream()
#         self.stream.close()
#         self.p.terminate()

class SpectrogramWidget(pg.PlotWidget):
    read_collected = QtCore.pyqtSignal(np.ndarray)
    def __init__(self):
        super(SpectrogramWidget, self).__init__()

        self.img = pg.ImageItem()
        self.addItem(self.img)

        self.img_array = np.zeros((1000, CHUNKSZ/2+1))

        # bipolar colormap

        pos = np.array([0., 1., 0.5, 0.25, 0.75])
        color = np.array([[0,255,255,255], [255,255,0,255], [0,0,0,255], (0, 0, 255, 255), (255, 0, 0, 255)], dtype=np.ubyte)
        cmap = pg.ColorMap(pos, color)
        lut = cmap.getLookupTable(0.0, 1.0, 256)

        self.img.setLookupTable(lut)
        self.img.setLevels([-50,40])

        freq = np.arange((CHUNKSZ/2)+1)/(float(CHUNKSZ)/FS)
        yscale = 1.0/(self.img_array.shape[1]/freq[-1])
        self.img.scale((1./FS)*CHUNKSZ, yscale)

        self.setLabel('left', 'Frequency', units='Hz')

        self.win = np.hanning(CHUNKSZ)
        self.show()

    def update(self, chunk):
        
        # normalized, windowed frequencies in data chunk
        
        data = chunk*self.win
        
        spec = np.fft.rfft(data.astype(np.float32))
        
        x_gpu = gpuarray.to_gpu(data.astype(np.float32))
        xf_gpu = gpuarray.empty(CHUNKSZ/2+1, np.complex64)
        plan_forward = cu_fft.Plan(x_gpu.shape, np.float32, np.complex64)
        cu_fft.fft(x_gpu, xf_gpu, plan_forward)
        
        print spec
        print xf_gpu.get()
        print len(spec)
        print len(xf_gpu.get())
        print np.allclose(spec[0:CHUNKSZ/2+1], xf_gpu.get(), atol=1)     
        

        # get magnitude
        psd = abs(spec/CHUNKSZ)
        
        # convert to dB scale
        psd = 20 * np.log10(psd)

        # roll down one and replace leading edge with new data

        self.img_array = np.roll(self.img_array, -1, 0)
        self.img_array[-1:] = psd

        self.img.setImage(self.img_array, autoLevels=False)

if __name__ == '__main__':
    app = QtGui.QApplication([])
    w = SpectrogramWidget()
    w.read_collected.connect(w.update)

    IN = recorder(w.read_collected)

    # time (seconds) between reads

    interval = FS/CHUNKSZ
    t = QtCore.QTimer()
    t.timeout.connect(IN.read)
    t.start(1000/interval) #QTimer takes ms


    app.exec_()
#     mic.close()

Overwriting spectrum.py
